In [1]:
from pyDeepInsight import ImageTransformer, LogScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
import pandas as pd
import numpy as np
import tensorflow.keras
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras import optimizers
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report
from scipy import linalg as scipy_linalg

from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.ensemble import AdaBoostClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score

import itertools


In [2]:
prosody_data = pd.read_csv('Prosodic_Features_MRH.csv',sep=",")
#expr = pd.read_csv(expr_file, sep="\t")
y = prosody_data['Hired3C'].values
#36
#X =prosody_data[['duration', 'energy', 'power', 'min_pitch', 'max_pitch', 'mean_pitch', 'pitch_sd', 'pitch_abs', 'pitch_quant', 'pitchUvsVRatio', 'Time:8', 'iDifference', 'diffPitchMaxMin', 'diffPitchMaxMean', 'diffPitchMaxMode', 'intensityMin', 'intensityMax', 'intensityMean', 'intensitySD', 'intensityQuant', 'diffIntMaxMin', 'diffIntMaxMean', 'diffIntMaxMode', 'avgVal1', 'avgVal2', 'avgVal3', 'avgBand1', 'avgBand2', 'avgBand3', 'fmean1', 'fmean2', 'fmean3', 'f2meanf1', 'f3meanf1', 'f1STD', 'f2STD']]
#49
X =prosody_data[['duration', 'energy', 'power', 'min_pitch', 'max_pitch', 'mean_pitch', 'pitch_sd', 'pitch_abs', 'pitch_quant', 'pitchUvsVRatio', 'Time:8', 'iDifference', 'diffPitchMaxMin', 'diffPitchMaxMean', 'diffPitchMaxMode', 'intensityMin', 'intensityMax', 'intensityMean', 'intensitySD', 'intensityQuant', 'diffIntMaxMin', 'diffIntMaxMean', 'diffIntMaxMode', 'avgVal1', 'avgVal2', 'avgVal3', 'avgBand1', 'avgBand2', 'avgBand3', 'fmean1', 'fmean2', 'fmean3', 'f2meanf1', 'f3meanf1', 'f1STD', 'f2STD', 'f3STD', 'f2STDf1', 'f2STDf2', 'jitter', 'shimmer', 'jitterRap', 'meanPeriod', 'percentUnvoiced', 'numVoiceBreaks', 'PercentBreaks', 'speakRate', 'numPause', 'maxDurPause']]
#25X = prosody_data[['pitch_quant', 'meanPeriod', 'mean_pitch', 'fmean3', 'f3STD', 'f2STD', 'pitch_sd', 'shimmer', 'avgBand2', 'intensityMean', 'jitter', 'intensitySD', 'intensityQuant', 'jitterRap', 'f1STD', 'avgVal2', 'fmean2', 'PercentBreaks', 'AvgTotFall:3', 'f2STDf1', 'pitch_abs', 'f2STDf2', 'avgBand1', 'percentUnvoiced', 'AvgTotRis:3']]
#X = prosody_data.iloc[:, 4:].values



In [3]:
def mahalanobis(x=None, data=None, cov=None):
    """Compute the Mahalanobis Distance between each row of x and the data  
    x    : vector or matrix of data with, say, p columns.
    data : ndarray of the distribution from which Mahalanobis distance of each observation of x is to be computed.
    cov  : covariance matrix (p x p) of the distribution. If None, will be computed from data.
    """
    x_minus_mu = x - np.mean(data)
    if not cov:
        cov = np.cov(data.values.T)
    inv_covmat = scipy_linalg.inv(cov)
    left_term = np.dot(x_minus_mu, inv_covmat)
    mahal = np.dot(left_term, x_minus_mu.T)
    return mahal.diagonal()

In [4]:
df_x =prosody_data[['duration', 'energy', 'power', 'min_pitch', 'max_pitch', 'mean_pitch', 'pitch_sd', 'pitch_abs', 'pitch_quant', 'pitchUvsVRatio', 'Time:8', 'iDifference', 'diffPitchMaxMin', 'diffPitchMaxMean', 'diffPitchMaxMode', 'intensityMin', 'intensityMax', 'intensityMean', 'intensitySD', 'intensityQuant', 'diffIntMaxMin', 'diffIntMaxMean', 'diffIntMaxMode', 'avgVal1', 'avgVal2', 'avgVal3', 'avgBand1', 'avgBand2', 'avgBand3', 'fmean1', 'fmean2', 'fmean3', 'f2meanf1', 'f3meanf1', 'f1STD', 'f2STD', 'f3STD', 'f2STDf1', 'f2STDf2', 'jitter', 'shimmer', 'jitterRap', 'meanPeriod', 'percentUnvoiced', 'numVoiceBreaks', 'PercentBreaks', 'speakRate', 'numPause', 'maxDurPause']]

df_x['mahala'] = mahalanobis(x=df_x, data=prosody_data[['duration', 'energy', 'power', 'min_pitch', 'max_pitch', 'mean_pitch', 'pitch_sd', 'pitch_abs', 'pitch_quant', 'pitchUvsVRatio', 'Time:8', 'iDifference', 'diffPitchMaxMin', 'diffPitchMaxMean', 'diffPitchMaxMode', 'intensityMin', 'intensityMax', 'intensityMean', 'intensitySD', 'intensityQuant', 'diffIntMaxMin', 'diffIntMaxMean', 'diffIntMaxMode', 'avgVal1', 'avgVal2', 'avgVal3', 'avgBand1', 'avgBand2', 'avgBand3', 'fmean1', 'fmean2', 'fmean3', 'f2meanf1', 'f3meanf1', 'f1STD', 'f2STD', 'f3STD', 'f2STDf1', 'f2STDf2', 'jitter', 'shimmer', 'jitterRap', 'meanPeriod', 'percentUnvoiced', 'numVoiceBreaks', 'PercentBreaks', 'speakRate', 'numPause', 'maxDurPause']])
#df_x =X

#df_x['mahala'] = mahalanobis(x=X, data=X)
X=df_x

<ipython-input-4-f4bb98b640ec>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_x['mahala'] = mahalanobis(x=df_x, data=prosody_data[['duration', 'energy', 'power', 'min_pitch', 'max_pitch', 'mean_pitch', 'pitch_sd', 'pitch_abs', 'pitch_quant', 'pitchUvsVRatio', 'Time:8', 'iDifference', 'diffPitchMaxMin', 'diffPitchMaxMean', 'diffPitchMaxMode', 'intensityMin', 'intensityMax', 'intensityMean', 'intensitySD', 'intensityQuant', 'diffIntMaxMin', 'diffIntMaxMean', 'diffIntMaxMode', 'avgVal1', 'avgVal2', 'avgVal3', 'avgBand1', 'avgBand2', 'avgBand3', 'fmean1', 'fmean2', 'fmean3', 'f2meanf1', 'f3meanf1', 'f1STD', 'f2STD', 'f3STD', 'f2STDf1', 'f2STDf2', 'jitter', 'shimmer', 'jitterRap', 'meanPeriod', 'percentUnvoiced', 'numVoiceBreaks', 'PercentBreaks'

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=23, stratify=y)
y_train


In [13]:
def CreateModel( X_train, X_test, y_train, y_test):
    y_01_test = []
    for i in y_test:
        if(i == 0):
            y_01_test.append(0)
        else:
            y_01_test.append(1)

    y_01_train = []
    for i in y_train:
        if(i == 0):
            y_01_train.append(0)
        else:
            y_01_train.append(1)

    X = X_train
    y = y_01_train
    clf = AdaBoostClassifier(n_estimators=100, random_state=0)
    clf.fit(X, y)
    clf.score(X, y)
    #AdaBoostClassifier(n_estimators=100, random_state=0)
    y_01_predict=clf.predict(X_test)
    #target_names = ['class 0', 'class 1']      
    #print(classification_report(y_01_test,y_01_predict,target_names=target_names, zero_division=0))

    X_12_train = X_train.loc[y_train!= 0]
    X_12_test = X_test.loc[y_test!= 0]

    #select from y_train where y value is not 0
    y_12_train= []
    for i in y_train:
        if(i != 0):
            y_12_train.append(i)

    #select from y_test where y value is not 0
    y_12_test = []
    for i in y_test:
        if(i != 0):
            y_12_test.append(i)

    X = X_12_train
    y = y_12_train
    clf = AdaBoostClassifier(n_estimators=100, random_state=0)
    clf.fit(X, y)
    clf.score(X, y)
    y_12_predict = clf.predict(X_12_test)
    #target_names = ['class 1', 'class 2']      
    #print(classification_report(y_12_test,y_12_predict ,target_names=target_names, zero_division=0))

    y1=list(y_test)
    y2=list(y_01_predict)
    y3=list(y_12_predict)
    combined_Predictions = []
    k=0
    for (i,j) in zip(y1,y2):
        if(i!=0):
            combined_Predictions.append(y3[k])
            k=k+1
        else:
            combined_Predictions.append(j)
    #target_names = ['class 0', 'class 1', 'class 2']
    #print(classification_report(y_test,combined_Predictions ,target_names=target_names, zero_division=0))
    acc = accuracy_score(y_test, combined_Predictions, normalize=True)

    return acc 


In [14]:
skf = StratifiedKFold(n_splits=5,shuffle=True, random_state=12)
cvscores =[]

for train_index, test_index in skf.split(X, y):

    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    
    scores = CreateModel( X_train, X_test, y_train, y_test)
    
    cvscores.append(scores)
   
print("CV Score: %.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))


TRAIN: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  15  16  19  20
  21  22  23  24  25  26  27  28  29  30  31  34  35  39  42  43  44  45
  46  47  49  50  51  53  54  55  56  57  58  59  60  61  64  66  67  68
  70  71  72  74  75  76  77  78  80  81  82  86  90  91  92  93  94  96
  97  98 100 101 102 103 104 106 107 108 110 114 115 116 117 118 120 121
 123 124 125 127 128 130 131 133 134 136 137 138 139 140 141 142 143 144
 145 146 148 149 150 152 153 154 155 156 157 158 159 160 161 162 164 165
 166 167 169 170 171 172 173 174 175 176 177 178 179 180 183 184 185 186
 187 188 189 190 191 193 196 197 198 199 201 203 205 206 209 210 211 213
 214 215 216 217 218 219 220 221 223 224 225 227 228 229 230 232 233 234
 235 236 238 240 241 242 243 245 246 247 248 250 252 253 254 256 257 258
 259 260 262 263 264 265 269 271 272 273 274 276 277 278 280 281 282 283
 284 285 286 287 288 289 290 291 294 297 298 299 300 301 302 303 305 306
 307 308 309 310 311 312 313 314 315 316 318

TRAIN: [  0   1   2   4   7   9  10  11  13  14  15  16  17  18  19  21  22  23
  24  25  26  27  28  30  31  32  33  34  36  37  38  39  40  41  42  43
  44  45  46  47  48  51  52  53  54  58  59  60  62  63  64  65  66  67
  69  70  73  76  78  79  80  82  83  84  85  86  87  88  89  90  91  92
  95  96  97  98  99 100 101 102 104 105 106 107 108 109 111 112 113 115
 116 117 118 119 121 122 123 124 125 126 127 129 132 133 135 136 137 138
 139 140 143 145 146 147 149 150 151 152 153 154 156 157 158 159 160 161
 162 163 164 165 166 168 169 170 171 172 174 176 178 179 180 181 182 183
 184 185 186 187 188 189 190 191 192 194 195 196 197 198 199 200 201 202
 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 219 221 222
 224 225 226 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242
 243 244 245 246 247 249 251 252 255 256 259 260 261 262 263 264 265 266
 267 268 269 270 272 273 275 277 278 279 280 281 282 283 284 286 287 289
 290 291 292 293 295 296 297 299 300 301 302